In [ ]:
import string
import math

input_file = '/content/drive/MyDrive/brown_corpus (1).txt'
num_words = 20000

with open(input_file, 'r') as file:
    text = file.read()
    words = text.split()[:num_words]

corpus_words = [word.replace(" ", "") for word in words]
user_text = ''.join(corpus_words)
user_text = user_text.translate(str.maketrans('', '', string.punctuation))
user_text = user_text.lower()

In [ ]:
import math
import string
# from nltk.corpus import brown
# import nltk
# nltk.download('brown')
from collections import Counter
import csv
from tqdm import tqdm

In [ ]:
def calculate_entropy(text, word, flag, expected_entropy,n):
    word_count = Counter()
    next_conditional_count = Counter()
    prev_conditional_count = Counter()
    next_entropy = 0
    prev_entropy = 0

    for i in range(len(text) - len(word) + 1):
        if text[i:i + len(word)] == word:
            word_count[word] += 1
            if i > 0:

                prev_letter = text[i-1]
                prev_conditional_count[prev_letter] += 1
            if i + len(word) < len(text):

                next_letter = text[i + len(word)]
                next_conditional_count[next_letter] += 1


    p_word = word_count[word] / (len(text) - len(word) + 1)
    ab = word_count[word]
    if ab == 1 :
      next_entropy = 0
    else:
      for letter, count in next_conditional_count.items():
        p_letter_given_word = count / word_count[word]
        next_entropy -= p_letter_given_word * math.log2(p_letter_given_word)

      for letter, count in prev_conditional_count.items():
        p_letter_given_word = count / word_count[word]
        prev_entropy -= p_letter_given_word * math.log2(p_letter_given_word)
    entropy = next_entropy + prev_entropy
    entropy = entropy/pow(expected_entropy[len(word)-1],n)
    # entropy = entropy*pow(ab,-0.05)
    if flag == 0:
      return entropy
    else:
      return entropy,ab

In [ ]:
expected_freq = [4353.447197068354, 470.7861382840183, 88.20185347830038, 21.433613979829794, 8.645264847512038, 4.402510675671583, 2.9410945213362005, 2.1966594990762895, 1.7545958387595022, 1.4846149186831825, 1.320845573760562, 1.2223719676549865, 1.1535782428298857, 1.1015627366061964, 1.0642357430570277, 1.045504103697871, 1.033132438898816, 1.0244859020563917, 1.0187164965625852, 1.0140070868287956, 1.010448529634453]

In [ ]:
expected_entropy = [3.615104146600215, 3.008971357041769, 2.1728871407856745, 1.6655986735467678, 1.5422606086250967, 1.4737354088570376, 1.4328895407724902, 1.3750568412434874, 1.324382076832141, 1.2878516947155783, 1.289154038818358, 1.2883972581433791, 1.29980950398726, 1.2939712293580952, 1.1857869690937333, 1.165041477635555, 1.1912020472935745, 1.1673555087603622, 1.1788479517716468, 1.1656756883115746, 1.1680495590016975, 1.1497904998059758, 1.1501955000865387]

In [ ]:
for i in range(20):
  expected_entropy.append(1.1501955000865387)
  expected_freq.append(1.010448529634453)

In [ ]:
def find_words(text, expected_entropy,n, words_entropy, words_freq):
    words = []

    i = 0
    with tqdm(total=len(text), desc="Processing") as pbar:
      while i < len(text):
          pbar.update(1)
          max_entropy = 0.0
          word_identified = False

          for j in range(i+1, min(i + 14, len(text))):
              word = text[i:j+1]
              word_length = len(word)
              prev_word = text[i:j]
              next_word = text[i:min(j+2,len(text)-1)]
              if word in words_entropy.keys():
                entropy = words_entropy[word]
              else:
                entropy,freq = calculate_entropy(text, word,1,expected_entropy,n)
                words_entropy[word] = entropy
                words_freq[word] = freq
              if next_word in words_entropy.keys():
                next_entropy = words_entropy[next_word]
              else:
                next_entropy,next_freq = calculate_entropy(text, next_word,1,expected_entropy,n)
                words_entropy[next_word] = next_entropy
                words_freq[next_word] = next_freq
              if prev_word in words_entropy.keys():
                prev_entropy = words_entropy[prev_word]
              else:
                prev_entropy,prev_freq = calculate_entropy(text, prev_word,1,expected_entropy,n)
                words_entropy[prev_word] = prev_entropy
                words_freq[prev_word] = prev_freq


              if entropy >= prev_entropy and entropy >= next_entropy :
                  max_entropy_index = j
                  word_identified = True
                  break
              # if len(word) == 2 and entropy*1.414 > prev_entropy and entropy*1.414 > next_entropy*1.732:
              #     max_entropy_index = j
              #     word_identified = True
              #     break

          if word_identified:
              words.append(text[i:max_entropy_index+1])
              i = max_entropy_index + 1
          else:
              words.append(text[i])
              i += 1


    return words, words_entropy, words_freq

In [ ]:
def optimize_words(identified_words, text, expected_entropy,n, words_entropy, words_freq):

        i = 0
        while i < len(identified_words) :

          word = identified_words[i]
          if word in words_entropy.keys():
              entropy = words_entropy[word]
              freq = words_freq[word]
          else:
              entropy,freq = calculate_entropy(text, word,1,expected_entropy,n)
              words_entropy[word] = entropy
              words_freq[word] = freq

          if entropy == 0 and freq > 1:
            i = i + 1
            continue
          else:
            index  = 0
            best_MI = -1
            word_length = len(word)

            for split_point in range(1, word_length):

                part_A = word[:split_point]
                part_B = word[split_point:]


                if part_A in words_entropy.keys():
                    entA = words_entropy[part_A]
                    freqA = words_freq[part_A]
                else:
                    entA,freqA = calculate_entropy(text, part_A,1,expected_entropy,n)
                    words_entropy[part_A] = entA
                    words_freq[part_A] = freqA

                if part_B in words_entropy.keys():
                    entB = words_entropy[part_B]
                    freqB = words_freq[part_B]
                else:
                    entB,freqB = calculate_entropy(text, part_B,1,expected_entropy,n)
                    words_entropy[part_B] = entB
                    words_freq[part_B] = freqB
                if entA > entropy and entB > entropy:

                    if entA + entB - entropy > best_MI and entA >0 and entB>0:

                      index = split_point
                      best_MI = entA + entB - entropy
                      bA = part_A
                      bB = part_B

            if index == 0:
                i = i + 1
                continue
            else :
                identified_words = replace_index(identified_words, i, bA)
                identified_words = add_index(identified_words, i+1, bB)
                i = i + 2
        return identified_words, words_entropy, words_freq

In [ ]:
def add_index(arr, index, value):
    if index < 0 or index > len(arr):
        print("Index out of range!")
        return arr

    new_arr = arr.copy()  # Create a copy of the original array to avoid modifying it directly
    new_arr.insert(index, value)
    return new_arr

def remove_index(arr, index):
    if index < 0 or index >= len(arr):
        print("Index out of range!")
        return arr

    new_arr = arr.copy()  # Create a copy of the original array to avoid modifying it directly
    del new_arr[index]
    return new_arr

def replace_index(arr, index, new_string):
    if index < 0 or index >= len(arr):
        print("Index out of range!")
        return arr

    new_arr = arr.copy()  # Create a copy of the original array to avoid modifying it directly
    new_arr[index] = new_string
    return new_arr

In [ ]:
def add_consequent_words(text, identified_words, expected_entropy,expected_freq,n, words_entropy, words_freq):
    i = 0
    new_words = []
    with tqdm(total=len(identified_words) - 2, desc="Adding Consequent Words") as pbar:
      while i < len(identified_words) :
          pbar.update(1)
          new_word = ""
          j = i
          word = identified_words[i]
          best_index = i
          new_word_1 = identified_words[i]
          if i == len(identified_words) -1:
            next_word_1 = new_word_1
          else:
            next_word_1 = new_word_1 + identified_words[i+1]
          new_entropy_1, new_freq_1 = calculate_entropy(text, new_word_1,1,expected_entropy,n)
          next_entropy_1, next_freq_1 = calculate_entropy(text, next_word_1,1,expected_entropy,n)
          if len(new_word_1)<20:
              new_freq_1 = new_freq_1/expected_freq[len(new_word_1)-1]
          if len(next_word_1) < 20:
              next_freq_1 = next_freq_1/expected_freq[len(next_word_1)-1]
          best_diff = new_entropy_1*new_freq_1*math.log(len(new_word_1)+1,2) - next_entropy_1*next_freq_1*math.log(len(next_word_1)+1,2)
          diff = 0
          # if len(word) > 23:
          #     current_entropy = pow(expected_entropy[23],1.75)
          # else:
          #     current_entropy = pow(expected_entropy[len(word)-1],1.75)
          # ent = calculate_entropy(text, word, 0) / current_entropy
          while j < min(i + 7, len(identified_words)):
              prev_word = new_word
              new_word += identified_words[j]
              if j == len(identified_words)-1:
                  next_word = new_word
              else:
                  next_word = new_word + identified_words[j+1]
              if len(new_word) < 15:
                  if prev_word == "":
                      prev_entropy = 0
                      prev_freq = 0
                  else:
                      if prev_word in words_entropy.keys():
                        prev_entropy = words_entropy[prev_word] + len(prev_word)*new_words.count(prev_word)*0.1
                        prev_freq = words_freq[prev_word]
                      else:
                        prev_entropy,prev_freq = calculate_entropy(text, prev_word,1,expected_entropy,n)
                        words_entropy[prev_word] = prev_entropy
                        words_freq[prev_word] = prev_freq
                  if new_word in words_entropy.keys():
                    new_entropy = words_entropy[new_word] + len(new_word)*new_words.count(new_word)*0.1
                    new_freq = words_freq[new_word]
                  else:
                    new_entropy,new_freq = calculate_entropy(text, new_word,1,expected_entropy,n)
                    words_entropy[new_word] = new_entropy
                    words_freq[new_word] = new_freq
                  if len(next_word) > 20:
                    next_entropy = 0
                    next_freq = 0
                  else:
                    if next_word in words_entropy.keys():
                      next_entropy = words_entropy[next_word] +  len(next_word)*new_words.count(next_word)*0.1
                      next_freq = words_freq[next_word]
                    else:
                      next_entropy,next_freq = calculate_entropy(text, next_word,1,expected_entropy,n)
                      words_entropy[next_word] = next_entropy
                      words_freq[next_word] = next_freq
                  if len(new_word)<20:
                    new_freq = new_freq/expected_freq[len(new_word)-1]
                  if len(prev_word)<20:
                    prev_freq = prev_freq/expected_freq[len(prev_word)-1]
                  if len(next_word) < 20:
                    next_freq = next_freq/expected_freq[len(next_word)-1]
                  if len(prev_word)<1:
                      ab = 1
                  else:
                      ab = len(prev_word)
                  # word_wasted = identified_words[j]
                  # if word_wasted in words_entropy.keys():
                  #    ww_ent = words_entropy[word_wasted]
                  # else:
                  #    ww_ent,ww_freq = calculate_entropy(text, word_wasted,1,expected_entropy,n)
                  #    words_entropy[word_wasted] = ww_ent
                  #    words_freq[word_wasted] = ww_freq
                  # words_wasted_entropy += ww_ent*len(word_wasted)
                  if new_entropy > prev_entropy and new_entropy > next_entropy: #and new_entropy*len(new_word)>words_wasted_entropy:        #Adding this condition makes normal words not follow this condition and causing issues. Check
                    diff = new_entropy - next_entropy
                  else:
                      diff = 0
                  if diff >= best_diff:
                      best_diff = diff
                      best_index = j
                      word = new_word

              j += 1

          new_words.append(word)
          i = best_index + 1

    return new_words, words_entropy, words_freq

In [ ]:
import os
def save_to_txt(identified_words,i):
   file_path = "D_10_" + str(i) + ".txt"
   data = identified_words

   if os.path.exists(file_path):
     os.remove(file_path)
   with open(file_path, 'w') as file:
     for word in data:
        file.write(word + '\n')

In [ ]:
import os
n = 0
def iterations(text, expected_entropy,expected_freq):
   dist =[]
   total_entropy =[]
   words_entropy = {'harish' : 1}
   words_freq = {'harish' : 1}
   words_dict = words_info = {
    'harish': {'freq': 0, 'ent':0, 'last_seen': 0}}
   for i in range(8):
     identified_words = []
     identified1_words = []
     identified2_words = []
     n = 1.0
     if i == 0:
       identified_words, words_entropy, words_freq = find_words(text, expected_entropy, n, words_entropy, words_freq)
       print(identified_words)
       #print("Distance ",dist(text,identified_words))
      #  dist.append(distance(text,identified_words))
       save_to_txt(identified_words,i)
       for word in identified_words:
        if word in words_dict.keys() and words_dict[word]['last_seen'] != i:
          words_dict[word]["freq"] += 1
          words_dict[word]['last_seen'] = i
        else:
          words_dict[word] = { "freq": 1, "ent": words_entropy[word], "last_seen":i }
       tot_ent = 0
       for word in identified_words:
         tot_ent += (words_entropy[word])/expected_freq[len(word)]
       total_entropy.append(tot_ent)
     else:
       file_path = 'D_10_' + str(i-1)+'.txt'
       with open(file_path, 'r') as file:
            identified_words = [line.strip() for line in file]

       identified1_words, words_entropy, words_freq = optimize_words(identified_words, text, expected_entropy, n,words_entropy, words_freq)
       print(identified1_words)
       identified2_words, words_entropy, words_freq = add_consequent_words(text, identified1_words, expected_entropy,expected_freq,n, words_entropy, words_freq)
       #print("Distance ",dist(text,identified2_words))
       print(identified2_words)
       for word in identified2_words:
        if word in words_dict.keys() and words_dict[word]['last_seen'] != i:
          words_dict[word]["freq"] += 1
          words_dict[word]['last_seen'] = i
        else:
          words_dict[word] = { "freq": 1, "ent": words_entropy[word], "last_seen":i }
      #  dist.append(distance(text,identified1_words))
       tot_en = 0
       for word in identified1_words:
         tot_en += (words_entropy[word])/expected_freq[len(word)]
       total_entropy.append(tot_en)
      #  dist.append(distance(text,identified2_words))
       save_to_txt(identified2_words,i)
       tot_ent = 0
       for word in identified2_words:
         tot_ent += (words_entropy[word])/expected_freq[len(word)]
       total_entropy.append(tot_ent)
   return total_entropy,words_dict

In [ ]:
dist = total_entropy =[]
words_dict = {}
total_entropy,words_dict = iterations(user_text, expected_entropy,expected_freq)

Processing:  32%|███▏      | 28472/88323 [04:23<09:13, 108.21it/s]


['the', 'ful', 'ton', 'county', 'gran', 'dju', 'ry', 'sa', 'idfrid', 'aya', 'nin', 'ves', 'tigat', 'ion', 'ofa', 'tla', 'nta', 'sre', 'ce', 'nt', 'pri', 'mar', 'yelect', 'ion', 'pro', 'duce', 'dno', 'evi', 'de', 'nce', 'that', 'any', 'irre', 'gu', 'lar', 'iti', 'esto', 'okpl', 'ace', 'the', 'jury', 'further', 'sa', 'idin', 'term', 'end', 'pres', 'ent', 'me', 'nts', 'that', 'the', 'ci', 'ty', 'executive', 'comm', 'it', 'te', 'ewhi', 'ch', 'had', 'over', 'all', 'char', 'ge', 'of', 'thee', 'le', 'ction', 'des', 'er', 'ves', 'the', 'pra', 'ise', 'and', 'than', 'ks', 'of', 'the', 'ci', 'ty', 'ofa', 'tla', 'nta', 'for', 'them', 'ann', 'erin', 'whi', 'ch', 'thee', 'le', 'ction', 'was', 'con', 'duct', 'ed', 'the', 'se', 'ptemb', 'er', 'octo', 'ber', 'term', 'jury', 'had', 'be', 'enc', 'har', 'ged', 'by', 'ful', 'ton', 'su', 'per', 'ior', 'court', 'judge', 'durwo', 'odpy', 'eto', 'in', 'ves', 'tigate', 'rep', 'or', 'tsof', 'pos', 'si', 'ble', 'irre', 'gu', 'lar', 'iti', 'es', 'int', 'he', 'hard

Adding Consequent Words:  88%|████████▊ | 27910/31651 [45:28<06:05, 10.23it/s]


['the', 'fulton', 'county', 'gran', 'd', 'jury', 'sa', 'idfr', 'id', 'aya', 'nin', 'ves', 'ti', 'gation', 'ofatlanta', 'sre', 'cent', 'primary', 'election', 'pro', 'duce', 'd', 'no', 'evidence', 'that', 'any', 'i', 'rre', 'gu', 'lar', 'iti', 'esto', 'ok', 'place', 'the', 'jury', 'further', 'sa', 'idin', 'term', 'end', 'present', 'ments', 'thatthecity', 'executive', 'committe', 'ewhich', 'had', 'over', 'all', 'charge', 'of', 'thee', 'lection', 'des', 'er', 'ves', 'the', 'pra', 'ise', 'and', 'than', 'ks', 'ofthe', 'city', 'ofatlanta', 'for', 'them', 'ann', 'erin', 'which', 'thee', 'lection', 'was', 'con', 'duct', 'edthe', 'se', 'ptember', 'october', 'term', 'jury', 'had', 'be', 'enc', 'har', 'ged', 'by', 'fulton', 'super', 'i', 'or', 'court', 'judge', 'dur', 'wood', 'py', 'eto', 'in', 'ves', 'tigate', 'rep', 'or', 'tsof', 'possible', 'i', 'rre', 'gu', 'lar', 'ities', 'inthe', 'hard', 'f', 'ought', 'pri', 'mar', 'ywhich', 'was', 'won', 'by', 'mayor', 'no', 'minate', 'ivanallenjr', 'only',

Adding Consequent Words:  98%|█████████▊| 29013/29624 [21:41<00:27, 22.29it/s]


['the', 'fulton', 'county', 'gran', 'd', 'jury', 'said', 'fr', 'id', 'aya', 'nin', 'vestigation', 'ofatlanta', 'sre', 'cent', 'primary', 'election', 'pro', 'duce', 'd', 'no', 'evidence', 'that', 'any', 'i', 'rre', 'gu', 'lar', 'iti', 'esto', 'o', 'k', 'place', 'the', 'jury', 'further', 'sa', 'idin', 'term', 'end', 'present', 'ments', 'that', 'thecity', 'executive', 'committe', 'ewhich', 'had', 'over', 'all', 'charge', 'of', 'theelection', 'des', 'er', 'ves', 'the', 'pra', 'ise', 'and', 'than', 'ks', 'ofthe', 'city', 'ofatlanta', 'for', 'them', 'ann', 'erin', 'which', 'theelection', 'was', 'con', 'duct', 'edthe', 'se', 'pt', 'ember', 'oc', 'tober', 'term', 'jury', 'had', 'be', 'enc', 'har', 'ged', 'by', 'fulton', 'super', 'i', 'or', 'court', 'judge', 'd', 'ur', 'wood', 'p', 'y', 'eto', 'in', 'ves', 'tigate', 'rep', 'or', 'tsof', 'possible', 'i', 'rre', 'gu', 'lar', 'ities', 'inthe', 'hard', 'f', 'ought', 'pri', 'mar', 'ywhich', 'was', 'won', 'by', 'mayor', 'no', 'minate', 'ivanallenjr',

Adding Consequent Words:  96%|█████████▌| 28022/29332 [18:41<00:52, 24.99it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-2d936bee7708>", line 3, in <cell line: 0>
    total_entropy,words_dict = iterations(user_text, expected_entropy,expected_freq)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-12-7f9c36d5b296>", line 38, in iterations
    identified2_words, words_entropy, words_freq = add_consequent_words(text, identified1_words, expected_entropy,expected_freq,n, words_entropy, words_freq)
                                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-10-04922adfe1e4>", line 16, in add_consequent_words
    new_entropy_1, new_freq_1 = calculate_entropy(text, new_word_1,1,expected_entropy,n)
                      

In [ ]:
dist = total_entropy =[]
words_dict = {}
total_entropy,words_dict = iterations(user_text, expected_entropy,expected_freq)

Processing:  32%|███▏      | 14306/44194 [01:31<03:11, 155.71it/s]


['the', 'ful', 'ton', 'county', 'gran', 'dju', 'ry', 'sa', 'idfri', 'day', 'anin', 'ves', 'tigat', 'ion', 'ofa', 'tla', 'nta', 'sre', 'ce', 'nt', 'pri', 'mar', 'yelect', 'ion', 'pro', 'duce', 'dno', 'evi', 'de', 'nce', 'that', 'an', 'yirr', 'egulari', 'ties', 'to', 'okpl', 'ace', 'the', 'ju', 'ry', 'further', 'sa', 'idin', 'term', 'end', 'pre', 'sen', 'tments', 'that', 'the', 'ci', 'ty', 'executive', 'co', 'mmitt', 'ee', 'whi', 'ch', 'had', 'over', 'all', 'cha', 'rge', 'of', 'the', 'el', 'ect', 'ion', 'des', 'er', 'ves', 'the', 'praise', 'and', 'than', 'ks', 'of', 'the', 'ci', 'ty', 'ofa', 'tla', 'nta', 'for', 'them', 'ann', 'erin', 'whi', 'ch', 'the', 'el', 'ect', 'ion', 'was', 'con', 'duct', 'ed', 'the', 'se', 'ptemb', 'er', 'octo', 'ber', 'term', 'ju', 'ry', 'had', 'be', 'enc', 'har', 'ge', 'dby', 'ful', 'ton', 'su', 'per', 'ior', 'court', 'ju', 'dge', 'dur', 'woodpy', 'eto', 'in', 'ves', 'tigat', 'ere', 'po', 'rts', 'of', 'pos', 'sible', 'irre', 'gula', 'ri', 'ties', 'in', 'the', '

Adding Consequent Words:  86%|████████▋ | 14275/16518 [12:52<02:01, 18.49it/s]


['the', 'fulton', 'county', 'gran', 'djury', 'said', 'friday', 'anin', 'ves', 'ti', 'gation', 'ofatlanta', 'sre', 'cent', 'primary', 'election', 'pro', 'duce', 'd', 'no', 'evidence', 'that', 'an', 'yi', 'rr', 'egularities', 'to', 'ok', 'place', 'the', 'jury', 'further', 'sa', 'i', 'din', 'term', 'end', 'pre', 'sen', 'tments', 'thatthecity', 'executive', 'committee', 'which', 'had', 'over', 'all', 'cha', 'rge', 'ofthe', 'election', 'des', 'er', 'ves', 'the', 'praise', 'and', 'than', 'ks', 'ofthe', 'city', 'ofatlanta', 'for', 'them', 'ann', 'erin', 'which', 'the', 'election', 'was', 'con', 'du', 'cted', 'the', 'se', 'ptember', 'oc', 'to', 'ber', 'term', 'jury', 'had', 'be', 'enc', 'har', 'ge', 'd', 'by', 'fulton', 'su', 'per', 'i', 'or', 'court', 'judge', 'd', 'ur', 'wood', 'py', 'eto', 'in', 'ves', 'ti', 'gat', 'ere', 'po', 'rts', 'of', 'possible', 'irregularities', 'inthe', 'hard', 'f', 'ought', 'primary', 'which', 'was', 'w', 'onby', 'mayor', 'no', 'min', 'ate', 'ivanallenjr', 'only',

Adding Consequent Words:  98%|█████████▊| 15276/15594 [06:32<00:08, 38.89it/s]


...

['the', 'fulton', 'county', 'gran', 'd', 'jury', 'said', 'friday', 'anin', 'ves', 't', 'igation', 'ofatlanta', 'sre', 'cent', 'primary', 'election', 'pro', 'duce', 'd', 'no', 'evidence', 'that', 'an', 'y', 'irregularities', 'to', 'o', 'k', 'place', 'the', 'jury', 'further', 'sa', 'i', 'din', 'term', 'end', 'pre', 'sen', 'tments', 'that', 'thecity', 'executive', 'committee', 'which', 'had', 'over', 'all', 'cha', 'rge', 'ofthe', 'election', 'des', 'er', 'ves', 'the', 'praise', 'and', 'than', 'ks', 'ofthe', 'city', 'ofatlanta', 'for', 'them', 'ann', 'erin', 'which', 'the', 'election', 'was', 'con', 'd', 'u', 'cted', 'the', 'se', 'ptember', 'oc', 'to', 'ber', 'term', 'jury', 'had', 'be', 'enc', 'har', 'ge', 'd', 'by', 'fulton', 'su', 'per', 'i', 'or', 'court', 'judge', 'd', 'ur', 'w', 'ood', 'p', 'y', 'eto', 'in', 'ves', 'ti', 'gat', 'ere', 'po', 'rts', 'of', 'possible', 'irregularities', 'inthe', 'hard', 'f', 'ought', 'primary', 'which', 'was', 'w', 'on', 'by', 'mayor', 'no', 'min', 'ate'

Adding Consequent Words: 100%|█████████▉| 15537/15579 [05:51<00:00, 44.18it/s]


['the', 'fulton', 'county', 'gran', 'd', 'jury', 'said', 'friday', 'anin', 'ves', 't', 'igation', 'ofatlanta', 'sre', 'cent', 'primary', 'election', 'pro', 'duce', 'd', 'no', 'evidence', 'that', 'an', 'y', 'irregularities', 'to', 'o', 'k', 'place', 'the', 'jury', 'further', 'sa', 'i', 'din', 'term', 'end', 'pre', 'sen', 'tments', 'that', 'thecity', 'executive', 'committee', 'which', 'had', 'over', 'all', 'cha', 'rge', 'ofthe', 'election', 'des', 'er', 'ves', 'the', 'praise', 'and', 'than', 'ks', 'ofthe', 'city', 'ofatlanta', 'for', 'them', 'ann', 'erin', 'which', 'the', 'election', 'was', 'con', 'd', 'u', 'cted', 'the', 'se', 'pt', 'ember', 'oc', 'to', 'ber', 'term', 'jury', 'had', 'be', 'enc', 'har', 'ge', 'd', 'by', 'fulton', 'su', 'per', 'i', 'or', 'court', 'judge', 'd', 'ur', 'w', 'ood', 'p', 'y', 'eto', 'in', 'ves', 'tigat', 'ere', 'po', 'rts', 'of', 'possible', 'irregularities', 'inthe', 'hard', 'f', 'ought', 'primary', 'which', 'was', 'w', 'on', 'by', 'mayor', 'no', 'min', 'ate'

Adding Consequent Words:  86%|████████▋ | 13445/15560 [05:08<00:48, 43.58it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-2d936bee7708>", line 3, in <cell line: 0>
    total_entropy,words_dict = iterations(user_text, expected_entropy,expected_freq)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-12-7f9c36d5b296>", line 38, in iterations
    identified2_words, words_entropy, words_freq = add_consequent_words(text, identified1_words, expected_entropy,expected_freq,n, words_entropy, words_freq)
                                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-10-04922adfe1e4>", line 17, in add_consequent_words
    next_entropy_1, next_freq_1 = calculate_entropy(text, next_word_1,1,expected_entropy,n)
                   